In [1]:
import SimpleITK as sitk
import mdai
import pandas as pd
import numpy as np
import cv2
import pydicom
import os
import multiprocessing as mp
import functools
from functools import partial
import mdai
import math
import time
import itertools
from pydicom.fileset import FileSet
from os import path as pathOs
from pathlib import Path
import toolz
from toolz.curried import pipe, map, filter, get
from toolz import curry
from os.path import basename, dirname, exists, isdir, join, split
import nnunetv2

import elastixRegister as elastixRegister
from elastixRegister import reg_a_to_b

from toolz.itertoolz import groupby
from toolz import curry
# import multiprocess
# p = multiprocess.Pool(os.cpu_count())
import multiprocessing as mp
import json
import os
from subprocess import Popen
import subprocess


In [3]:
inferred_example_path='/workspaces/konwersjaJsonData/explore/to_eval_test_cases/inferred/inferred_test_cases/9002600.nii.gz'
gold_standard_labels_path='/workspaces/konwersjaJsonData/explore/to_eval_test_cases/sources/Dataset287_Prostate/labelsTr/9002600.nii.gz'

inferred_example_img= sitk.ReadImage(inferred_example_path)
gold_standard_labels_img= sitk.ReadImage(gold_standard_labels_path)


In [ ]:
""" 
we identify all single lesion files associated with our inferred label 
and we calculate how many of them has at least 80% coverage in the inferred area
we will also calculate the percent of whole area that is not covered by any lesion
"""